# **딥러닝 무작정 따라하기**

# 1.환경준비

### (1) 라이브러리 Import

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [9]:
pip install torch

   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/204.1 MB 2.6 MB/s eta 0:01:18
   ---------------------------------------- 0.7/204.1 MB 7.4 MB/s eta 0:00:28
   ---------------------------------------- 1.2/204.1 MB 9.9 MB/s eta 0:00:21
   ---------------------------------------- 2.0/204.1 MB 10.5 MB/s eta 0:00:20
   ---------------------------------------- 2.5/204.1 MB 10.0 MB/s eta 0:00:21
    --------------------------------------- 3.2/204.1 MB 10.7 MB/s eta 0:00:19
    --------------------------------------- 3.8/204.1 MB 11.0 MB/s eta 0:00:19
    --------------------------------------- 4.3/204.1 MB 11.0 MB/s eta 0:00:19
    --------------------------------------- 4.9/204.1 MB 11.1 MB/s eta 0:00:18
   - -------------------------------------- 5.4/204.1 MB 11.2 MB/s eta 0:00:18
   - -------------------------------------- 6.0/204.1 MB 11.2 MB/s eta 0:00:18
   - -------------------------------------- 6.6/204.1 MB 11.4 M

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

### (2) 필요 함수 생성
* 아래 생성하는 함수들은 차차 배우게 됩니다.
* 지금은 함수 이름 정도만 눈여겨 봅시다.

* 딥러닝을 위한 데이터로더 만들기

In [ ]:
def make_DataSet(x_train, x_val, y_train, y_val, batch_size = 32) :

    # 데이터 텐서로 변환
    x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

    # TensorDataset 생성 : 텐서 데이터셋으로 합치기
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

    # DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)

    return train_loader, x_val_tensor, y_val_tensor

* 학습을 위한 함수

In [ ]:
def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)                  # 전체 데이터셋의 크기
    num_batches = len(dataloader)                   # 배치 크기
    tr_loss = 0
    model.train()                                   # 훈련 모드로 설정
    for batch, (X, y) in enumerate(dataloader):     # batch : 현재 배치 번호, (X, y) : 입력 데이터와 레이블
        X, y = X.to(device), y.to(device)           # 입력 데이터와 레이블을 지정된 장치(device, CPU 또는 GPU)로 연결

        # Feed Forward
        pred = model(X)
        loss = loss_fn(pred, y)
        tr_loss += loss

        # Backpropagation
        loss.backward()             # 역전파를 통해 모델의 각 파라미터에 대한 손실의 기울기를 계산
        optimizer.step()            # 옵티마이저가 계산된 기울기를 사용하여 모델의 파라미터를 업데이트
        optimizer.zero_grad()       # 옵티마이저의 기울기 값 초기화. 기울기가 누적되는 것 방지

    tr_loss /= num_batches          # 모든 배치에서의 loss 평균

    return tr_loss.item()

* 검증을 위한 함수

In [ ]:
def evaluate(x_val_tensor, y_val_tensor, model, loss_fn, device):
    model.eval()                        # 모델을 평가 모드로 설정

    with torch.no_grad():               # 평가 과정에서 기울기를 계산하지 않도록 설정(메모리 사용을 줄이고 평가 속도를 높입니다.)
        x, y = x_val_tensor.to(device), y_val_tensor.to(device)
        pred = model(x)
        eval_loss = loss_fn(pred, y).item()    # 예측 값 pred와 실제 값 y 사이의 손실 계산

    return eval_loss, pred

### (3) device 준비(cpu or gpu)

In [ ]:
# cpu 혹은 gpu 사용
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 2.Regression : Advertising

## (1) 데이터 전처리 - 기본

### 1) 데이터 준비

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/advertising.csv'
adv = pd.read_csv(path)
adv.head()

In [ ]:
target = 'Sales'
x = adv.drop(target, axis=1)
y = adv.loc[:, target]

### 2) 가변수화

### 3) 데이터분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2, random_state = 20)

### 4) 스케일링

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

## (2) 딥러닝 모델링

### 1) 딥러닝을 위한 데이터 준비

*  pandas 데이터프레임 ==> PyTorch의 DataLoader로 변환
    * 데이터 텐서로 변환
    * 텐서 데이터셋으로 합치기 : x, y
    * 데이터 로더 생성

* 1-(2) 에서 생성한 함수 : **make_DataLoader**

In [ ]:
train_loader, x_val_ts, y_val_ts = make_DataSet(x_train, x_val, y_train, y_val, batch_size = 4)

In [ ]:
# 첫번째 배치만 로딩해서 살펴보기
for x, y in train_loader:
    print(f"Shape of x [rows, columns]: {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

### 2) 모델 선언

In [ ]:
x.shape

In [ ]:
n_feature = x.shape[1]

# 모델 구조 설계
model = nn.Sequential(
            nn.Linear(n_feature, 3),    # hidden layer(input, output)
            nn.ReLU(),                  # 활성함수
            nn.Linear(3, 1)             # output layer
        ).to(device)                    # cpu, gpu 사용 설정. cpu인 경우 생략가능

print(model)

* Loss function과 Optimizer

In [ ]:
loss_fn = nn.MSELoss()          # MSE
optimizer = Adam(model.parameters(), lr=0.01)  # model.parameters() : 모델의 가중치와 편향

### 4) 학습

* 1-(2)에서 생성한 함수 : **train**, **evaluate**

In [ ]:
list(range(20))

In [ ]:
epochs = 20
for t in range(epochs):
    tr_loss = train(train_loader, model, loss_fn, optimizer, device)
    val_loss, _ = evaluate(x_val_ts, y_val_ts, model, loss_fn, device)

    print(f"Epoch {t+1}, train loss : {tr_loss:4f}, val loss : {val_loss:4f}")

### 5) 모델 평가

In [ ]:
loss, _ = evaluate(x_val_ts, y_val_ts, model, loss_fn, device)
loss

# 3.Regression : 보스턴 집값

## (1) 데이터 전처리 - 기본

### 1) 데이터 준비

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/boston.csv'
data = pd.read_csv(path)
data.head()

In [ ]:
target = 'medv'
x = data.drop(target, axis=1)
y = data.loc[:, target]

### 2) 가변수화

### 3) 데이터분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2, random_state = 20)

### 4) 스케일링

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

## (2) 딥러닝 모델링

### 1) 딥러닝을 위한 데이터 준비

*  pandas 데이터프레임 ==> PyTorch의 DataLoader로 변환
    * 데이터 텐서로 변환
    * 텐서 데이터셋으로 합치기 : x, y
    * 데이터 로더 생성

* 1-(2) 에서 생성한 함수 : **make_DataLoader**

In [ ]:
train_loader, x_val_ts, y_val_ts = make_DataSet(x_train, x_val, y_train, y_val, batch_size = 4)

In [ ]:
# 첫번째 배치만 로딩해서 살펴보기
for x, y in train_loader:
    print(f"Shape of x [rows, columns]: {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

### 2) 모델 선언

In [ ]:
n_feature = x.shape[1]

# 모델 구조 설계
model = nn.Sequential(

            ).to(device)

print(model)

* Loss function과 Optimizer

In [ ]:
loss_fn =
optimizer =

### 4) 학습

In [ ]:
epochs = 20
for t in range(epochs):
    tr_loss = train(train_loader, model, loss_fn, optimizer, device)
    val_loss, _ = evaluate(x_val_ts, y_val_ts, model, loss_fn, device)

    print(f"Epoch {t+1}, train loss : {tr_loss:4f}, val loss : {val_loss:4f}")

### 5) 모델 평가

In [ ]:
loss, _ = evaluate(x_val_ts, y_val_ts, model, loss_fn, device)